In [1]:
from scipy.stats import ttest_ind, normaltest, shapiro

split_n = 0
splits = {}
with open("ALL_classification_results.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        split_line = line.split()
        if split_line[0] == "Split":
            split_n = int(split_line[1])
            splits[split_n] = {}
        elif split_line[0] == "BERT":
            if split_line[1] == "short":
                splits[split_n]["BERT_short"] = (float(split_line[5][:-1]))
            else:
                splits[split_n]["BERT_long"] = (float(split_line[5][:-1]))
        elif split_line[0] == "CNN":
            splits[split_n]["CNN"] = (float(split_line[3][:-1]))

        elif split_line[0] == "GBC":
            splits[split_n]["GBC"] = (float(split_line[3][:-1]))

In [2]:
import pandas as pd
dt = pd.DataFrame(splits)

/tmp/ipykernel_3366502/227332269.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


CNN_acc = []
with open("CNN_out.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        split_line = line.split()
        CNN_acc.append(float(split_line[1][:-1]))
dt.loc["CNN"] = CNN_acc

In [15]:
dt

,0,1,2,3,4
BERT_short,0.828217,0.825440,0.831879,0.830659,0.832404
BERT_long,0.856964,0.863636,0.857920,0.877490,0.865866
CNN,0.813855,0.819875,0.842651,0.790576,0.806149
GBC,0.782554,0.814256,0.817467,0.770644,0.792008


In [3]:
_, p_bert_short = shapiro(dt.loc["BERT_short"])
_, p_bert_long = shapiro(dt.loc["BERT_long"])
_, p_cnn = shapiro(dt.loc["CNN"])
_, p_gbc = shapiro(dt.loc["GBC"])
print("Normality not rejected:", p_bert_short > 0.05, p_bert_long > 0.05, p_cnn > 0.05, p_gbc > 0.05)

Normality not rejected: True True True True


### In case normality assumption is met

In [17]:
#BERT vs CNN
ttest_ind((dt.loc["BERT_short"] + dt.loc["BERT_long"])/2, dt.loc["CNN"])

TtestResult(statistic=3.685461535616591, pvalue=0.006170403156304679, df=8.0)

In [4]:
#BERT vs CNN
ttest_ind( dt.loc["CNN"], (dt.loc["BERT_short"] + dt.loc["BERT_long"])/2)

TtestResult(statistic=-3.685461535616591, pvalue=0.006170403156304679, df=8.0)

In [18]:
#BERT vs GBC
ttest_ind((dt.loc["BERT_short"] + dt.loc["BERT_long"])/2, dt.loc["GBC"])

TtestResult(statistic=5.576694808646178, pvalue=0.0005243230314471253, df=8.0)

In [19]:
#CNN vs GBC
ttest_ind(dt.loc["CNN"], dt.loc["GBC"])

TtestResult(statistic=1.5460995759603404, pvalue=0.16066477744842128, df=8.0)